## Using helix_funcs

An example of how to use the helix_funcs module to process the HelixScope datafiles, and generate a master output table.

Process_files should generate a similarly named csv file in the process folder.
combine_processed 

In [2]:
import helix_funcs
import geopandas as gpd
import pandas as pd
# from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#s = gpd.read_file('./data/gadm28_adm1/gadm28_adm1.shp')
#s = s.to_crs(epsg='4326')
s = gpd.read_file("./data/gadm28_adm0_simplified/gadm28_adm0_simplified.shp")  # <--- ADMIN LEVEL 0
#s = gpd.read_file("./data/gadm28_adm1_simplified/gadm28_adm1_simplified.shp")   # <--- ADMIN LEVEL 1

In [ ]:
fs = helix_funcs.identify_netcdf_and_csv_files()

In [ ]:
fs['nc'][0:2]

In [ ]:
%%time
for f in fs['nc'][0:2]:
    helix_funcs.process_file(file=f, shps=s, admin_level=0, verbose=True)

### Speed up 

Looks like we achieve a 34 second execution per file (down from 2 mins) when we use a simplifed set of geometries.



## Investiage areas with no zonal stats

Seems like small areas are failing. Need to investigate best way to handle this.

* check simple shapes produce same number of elements as complex shapes - confimred
* check shapefile has same number of shapes as output file - confirmed
* check simple and complex processed results for same file show good agreement. - confirmed

* Add kwarg for ADMIN0 or Admin1. Have a differnt root folder for both in processed data.
* MOVE alredy processed ADMIN1 level data to the `processed/admin1/` directory - Failed. Columns changed. Will need to recalculate :(
* SIMPLIFY the ADMIN-0 level geometry file as I did with the ADMIN1 file. - confirmed
* RUN ADMIN0 for all data.

Perhaps we should have two tables: ADMIN0 level (based on simplified shapes), and ADMIN1 level.

The ADMIN 0 level makes sense to calculate (mean, min, max, stdev), as those shapes should be large. BUT for ADMIN 1 level, the shapes can be far smaller. In which case it doesnt make sense to calculate those stats for every shape.

* given time, we could work out which shapes it makes sense for (e.g. with counts > 3) and make some distinction.
* with short time (which is the case) it maybe only makes sense to offer mean over those areas (which in most cases will be the mean of 1 cell (i.e. simply the value of the gridcell).


THEREFORE:

1. Process all data to ADMIN 0 (using simplified country shapes)
    * use this as MVP
2. CALCULATE TIME RELATIVE STATISTICS
    * i.e. for datatets with Month distinctions calculate the seasonality
3. Attempt to do the ADMIN 1 level data.

# Deal with outputs

Reduce significant digits. Join together dataframes.

In [2]:
fs = helix_funcs.identify_netcdf_and_csv_files()

In [3]:
len(fs['csv'])

274

In [10]:
%%time
helix_funcs.combine_processed_results(path='./processed/admin0',
                                      table_name='./master_admin0.csv')  #<-- join all results files together into a master_admin1.csv

Made ./master_admin0.csv: 653,183 rows of data. 2,849 sources.
CPU times: user 29.4 s, sys: 980 ms, total: 30.3 s
Wall time: 39.7 s


In [11]:
tmp = pd.read_csv('./master_admin0.csv')

In [12]:
len(tmp)

653183

In [13]:
tmp.head()

,name_0,iso,variable,swl_info,count,max,min,mean,std,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal,season,is_monthly,month,run
0,Afghanistan,AFG,Soybeans_yield_perc_change,1.5,NaN,NaN,NaN,-3.1,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,5
1,Angola,AGO,Soybeans_yield_perc_change,1.5,NaN,NaN,NaN,-5.5,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,5
2,Albania,ALB,Soybeans_yield_perc_change,1.5,NaN,NaN,NaN,-2.0,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,5
3,Argentina,ARG,Soybeans_yield_perc_change,1.5,NaN,NaN,NaN,-2.4,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,5
4,Armenia,ARM,Soybeans_yield_perc_change,1.5,NaN,NaN,NaN,0.9,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,5


In [21]:
print(len(tmp['variable'].unique()))
for var in (tmp['impact_tag'].unique()):
    print(var)

25
ag
cl
w
eco


In [7]:
mask = tmp['variable'] == 'Rice_yield_perc_change' #'river_floods_PopAff' #"river_floods_ExpDam"
shorter = tmp[mask]

In [8]:
shorter.head()

,name_0,iso,variable,swl_info,count,max,min,mean,std,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal,season,is_monthly,month,run
135,Afghanistan,AFG,Rice_yield_perc_change,1.5,NaN,NaN,NaN,4.8,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,3
136,Angola,AGO,Rice_yield_perc_change,1.5,NaN,NaN,NaN,-2.0,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,3
137,Albania,ALB,Rice_yield_perc_change,1.5,NaN,NaN,NaN,4.8,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,3
138,Argentina,ARG,Rice_yield_perc_change,1.5,NaN,NaN,NaN,0.2,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,3
139,Australia,AUS,Rice_yield_perc_change,1.5,NaN,NaN,NaN,1.5,NaN,ag,NaN,NaN,NaN,ECEARTH,False,False,NaN,False,NaN,3


In [15]:
shorter['swl_info'].unique()

array([ 4.,  1.,  2.])

In [ ]:
tmp = pd.read_csv("./master_admin0.csv")
tmp.head()

In [ ]:
set(tmp['iso'].values)

In [ ]:
is_season = tmp['is_seasonal'] == True
print((len(tmp[is_season])/len(tmp)) * 100.,"% has seasonal data")

In [ ]:
is_season = tmp['is_monthly'] == True
print((len(tmp[is_season])/len(tmp)) * 100.,"% has monthly data")

In [ ]:
output_files = helix_funcs.identify_netcdf_and_csv_files('processed/')

In [ ]:
len(output_files['csv'])

In [ ]:
tmp = pd.read_csv(outputs['csv'][-1])

In [ ]:
country_mask = tmp['iso'] == 'ESP'

In [ ]:
tmp = tmp[country_mask].head()

In [ ]:
admin_mask = tmp['id_1'] == 1

In [ ]:
tmp['id_1'].unique()

## Test plots

We need to make an easy way to preview how a specific variables choropleths will appear.
This means read a file, and make a preview plot using geopandas.


In [ ]:
output_files = helix_funcs.identify_netcdf_and_csv_files('processed/')

In [ ]:
helix_funcs.map_file_by_iso(f=output_files['csv'][10], s=s, iso='FRA')

## SIMPLIFY LIFE!

Before passing the polygons for zonal analysis, they should be simplified. 
Looks like the call should be `stest.geometry.simplify(0.2, preserve_topology=False)`

The geometry files themsevels should be simplified, and read in already done. I have another notebook to simply (and correct) these data.

In [ ]:
stest.geometry.simplify(0.2, preserve_topology=False).plot()

In [ ]:
stest = s[s['iso'] == 'BRA']
stest.geometry.plot()

In [ ]:
stest.geometry.simplify(0.2, preserve_topology=True).plot()

## Next Step

Work out problem of small shapes (probably will need to have a logical test for small shapes, and buffer the geom before zonal stats are calculated). Perhaps this should even be done prior to any loop, initially when the shapes are calculaed.

First can look and see what admins are absent from file (to see where the problem lies).

Test with simplifed shapes.

#### REDUCE SIZE OF FINAL DATA BY removing significant digits from values
e.g. 10.6466969914 should be converted to simply 10.6





In [ ]:
def combine_processed_results(path='./processed/admin1',
                              table_name="./master_admin1.csv"):
    """Combine all the csv files in the path (e.g. all processed files)
    into a single master table
    """
    output_files = identify_netcdf_and_csv_files(path)
    frames = [pd.read_csv(csv_file) for csv_file in output_files['csv']]
    master_table = pd.concat(frames)
    master_table.to_csv(table_name, index=False)
    print("Made {0}: {1:,g} rows of data. {2:,g} sources.".format(table_name,
                                                        len(master_table),
                                                        len(output_files['csv'])
                                                                 ))
    return


In [ ]:
path='./processed'
output_files = helix_funcs.identify_netcdf_and_csv_files(path)
frames = [pd.read_csv(csv_file) for csv_file in output_files['csv'][0:10]]

In [ ]:
for  n, frame in enumerate(frames):
    print(n, min(frames[0]['min']), max(frames[0]['min']), min(frames[0]['max']), max(frames[0]['max']), )

In [ ]:
frames[0].head()

In [ ]:
test = frames[0].round(1)

In [ ]:
admin_level = 0

if admin_level == 0:
    admin_prefix = 'admin0/'
elif admin_level == 1:
    admin_prefix = 'admin1/'
else:
    raise ValueError("admin_level kwarg must be either 0 or 1")
print(admin_prefix)